In [ ]:
# ASSIGNMENT 5
# 122B1F073
# TYITB073
# Perform the following operations using Python on the Air  quality data sets a. Data cleaning b. Data transformation

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Desktop/FODS/Air_Quality.csv')

In [6]:
print(df.info())   # See data types, non-null counts
print(df.describe())  # Get summary stats for numeric columns


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16218 entries, 0 to 16217
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unique ID       16218 non-null  int64  
 1   Indicator ID    16218 non-null  int64  
 2   Name            16218 non-null  object 
 3   Measure         16218 non-null  object 
 4   Measure Info    16218 non-null  object 
 5   Geo Type Name   16218 non-null  object 
 6   Geo Join ID     16218 non-null  int64  
 7   Geo Place Name  16218 non-null  object 
 8   Time Period     16218 non-null  object 
 9   Start_Date      16218 non-null  object 
 10  Data Value      16218 non-null  float64
 11  Message         0 non-null      float64
dtypes: float64(2), int64(3), object(7)
memory usage: 1.5+ MB
None
           Unique ID  Indicator ID   Geo Join ID    Data Value  Message
count   16218.000000  16218.000000  1.621800e+04  16218.000000      0.0
mean   372730.417746    427.803613  6.097103e+05  

In [13]:
# 1. Remove the 'Message' column (empty column )
df_cleaned = df.drop(columns=['Message'])

In [14]:
# 2. Convert 'Start_Date' to datetime
df_cleaned['Start_Date'] = pd.to_datetime(df_cleaned['Start_Date'], errors='coerce')

In [15]:
# 4. Check for erroneous values (e.g., negative 'Data Value') and replace them with NaN
df_cleaned['Data Value'] = df_cleaned['Data Value'].apply(lambda x: np.nan if x < 0 else x)

In [16]:
# 6. Remove duplicates (if any exist)
df_cleaned = df_cleaned.drop_duplicates()

In [17]:
# Final overview of cleaned data
df_cleaned.info()  # Display summary information about the DataFrame
df_cleaned.head()  # Display the first few rows of the cleaned DataFrame

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16218 entries, 0 to 16217
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Unique ID       16218 non-null  int64         
 1   Indicator ID    16218 non-null  int64         
 2   Name            16218 non-null  object        
 3   Measure         16218 non-null  object        
 4   Measure Info    16218 non-null  object        
 5   Geo Type Name   16218 non-null  object        
 6   Geo Join ID     16218 non-null  int64         
 7   Geo Place Name  16218 non-null  object        
 8   Time Period     16218 non-null  object        
 9   Start_Date      16218 non-null  datetime64[ns]
 10  Data Value      16218 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 1.4+ MB


,Unique ID,Indicator ID,Name,Measure,Measure Info,Geo Type Name,Geo Join ID,Geo Place Name,Time Period,Start_Date,Data Value
0,172653,375,Nitrogen dioxide (NO2),Mean,ppb,UHF34,203,Bedford Stuyvesant - Crown Heights,Annual Average 2011,2010-12-01,25.30
1,172585,375,Nitrogen dioxide (NO2),Mean,ppb,UHF34,203,Bedford Stuyvesant - Crown Heights,Annual Average 2009,2008-12-01,26.93
2,336637,375,Nitrogen dioxide (NO2),Mean,ppb,UHF34,204,East New York,Annual Average 2015,2015-01-01,19.09
3,336622,375,Nitrogen dioxide (NO2),Mean,ppb,UHF34,103,Fordham - Bronx Pk,Annual Average 2015,2015-01-01,19.76
4,172582,375,Nitrogen dioxide (NO2),Mean,ppb,UHF34,104,Pelham - Throgs Neck,Annual Average 2009,2008-12-01,22.83


In [25]:
# Data Cleaning is over 


In [26]:
# 1. Load the cleaned dataset (assuming it's already cleaned)
df_transformed = df_cleaned.copy()

In [27]:
# 2. Create new features: Extract year and month from 'Start_Date'
df_transformed['Year'] = df_transformed['Start_Date'].dt.year
df_transformed['Month'] = df_transformed['Start_Date'].dt.month

In [28]:
# 3. Pivot table: Average 'Data Value' by 'Geo Place Name' and 'Year'
pivot_table = df_transformed.pivot_table(
    values='Data Value', 
    index='Geo Place Name', 
    columns='Year', 
    aggfunc='mean'
).reset_index()

In [29]:
# 4. Normalize 'Data Value' (scaling between 0 and 1)
df_transformed['Normalized Value'] = (
    (df_transformed['Data Value'] - df_transformed['Data Value'].min()) / 
    (df_transformed['Data Value'].max() - df_transformed['Data Value'].min())
)

In [30]:
# 5. Aggregation: Group data by 'Year' and calculate average 'Data Value'
yearly_avg = df_transformed.groupby('Year')['Data Value'].mean().reset_index()

In [31]:
# 6. Rename columns for better readability
df_transformed.rename(columns={'Geo Place Name': 'Location'}, inplace=True)

In [32]:
# Display transformed data
print("Pivot Table (Average Data Value by Location and Year):")
print(pivot_table.head())

print("\nYearly Average Data Value:")
print(yearly_avg.head())

print("\nTransformed Data (First 5 Rows):")
print(df_transformed.head())

Pivot Table (Average Data Value by Location and Year):
Year                      Geo Place Name       2005     2008       2009  \
0     Bay Ridge and Dyker Heights (CD10)  20.466667  18.6025  19.141250   
1                  Bayside - Little Neck  15.626667  16.8175  15.447222   
2      Bayside Little Neck-Fresh Meadows        NaN  17.4675  18.045000   
3         Bayside and Little Neck (CD11)  23.366667  17.0375  17.846250   
4               Bedford Stuyvesant (CD3)  17.266667  21.2925  19.948750   

Year       2010       2011       2012       2013       2014       2015  \
0     19.687143  14.008889  17.445714  17.554286  17.518000  15.536667   
1     18.327143  13.094444  12.942353  12.867000  12.328571  11.119545   
2     18.767143  16.814286  16.380000  17.010000  17.026000  14.758889   
3     18.511429  13.238889  16.101429  16.797143  16.934000  14.611111   
4     21.031429  15.114444  18.751429  18.880000  18.400000  16.677778   

Year      2016       2017       2018       2019  